In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import wav2vec2, bert, fastspeech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
# from malaya_speech.train.model.fastvc.model import Decoder

# config = malaya_speech.config.fastspeech_config
# dim = 768
# config['encoder_hidden_size'] = dim
# config['encoder_num_hidden_layers'] = 4
# config['encoder_num_attention_heads'] = 12
# config['encoder_intermediate_size'] = dim * config['encoder_num_hidden_layers']
# config = fastspeech.Config(vocab_size = 1, **config)
# encoder = Decoder(config.encoder_self_attention_params)

In [6]:
# class Encoder:
#     def __init__(self, config):
#         self.config = config
#         self.model = None
#     def __call__(self, x, input_mask, training = True):
#         if self.model is None:
#             input_mask = tf.logical_not(input_mask)
#             self.model = bert.BertModel(config = self.config, is_training = training,
#                                   input_ids = x, input_mask = input_mask)
#         return self.model.sequence_output
    
# encoder = Encoder(config = bert.BertConfig())

In [7]:
from malaya_speech.train.model.conformer.model import Model as ConformerModel

class Encoder:
    def __init__(self, config):
        self.config = config
        self.encoder = ConformerModel(**self.config)

    def __call__(self, x, input_mask, training = True):
        return self.encoder(x)
    
config_conformer = malaya_speech.config.conformer_base_encoder_config
config_conformer['subsampling']['type'] = 'none'
config_conformer['dropout'] = 0.0
encoder = Encoder(config_conformer)

In [8]:
cfg = wav2vec2.Wav2Vec2Config()
model = wav2vec2.Model(cfg, encoder)

In [9]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [10]:
r, num_vars, curr_temp = model(X, padding_mask = X_len)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Tensor("wav2vec2/dense_3/BiasAdd:0", shape=(?, ?, 256), dtype=float32) Tensor("wav2vec2/dense_2/BiasAdd:0", shape=(?, ?, 256), dtype=float32) Tensor("wav2vec2/transpose:0", shape=(100, ?, ?, 256), dtype=float32)
Instructions for updating:
dim is deprecated, use axis instead
Tensor("wav2vec2/strided_slice_11:0", shape=(101, ?, ?), dtype=float32) Tensor("wav2vec2/All:0", shape=(100, ?, ?), dtype=bool)


{'x': <tf.Tensor 'wav2vec2/concat_1:0' shape=(101, ?, ?) dtype=float32>,
 'padding_mask': <tf.Tensor 'wav2vec2/LogicalNot:0' shape=(?, ?) dtype=bool>,
 'features_pen': <tf.Tensor 'wav2vec2/Mean:0' shape=() dtype=float32>,
 'prob_perplexity': <tf.Tensor 'wav2vec2/GumbelVectorQuantizer/Sum_3:0' shape=() dtype=float32>,
 'code_perplexity': <tf.Tensor 'wav2vec2/GumbelVectorQuantizer/Sum_1:0' shape=() dtype=float32>}

In [11]:
logits = r['x']
logits = tf.transpose(logits, [2, 1, 0])
logits = tf.reshape(logits, (-1, tf.shape(logits)[-1]))
target = tf.zeros(shape = (tf.shape(r['x'])[1] * tf.shape(r['x'])[2]), dtype = tf.int32)
logits, target

(<tf.Tensor 'Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'zeros:0' shape=(?,) dtype=int32>)

In [12]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target, logits = logits)
loss = tf.reduce_sum(loss)

In [13]:
extra_losses = []

if 'prob_perplexity' in r:
    extra_losses.append(
        (num_vars - r["prob_perplexity"])
        / num_vars
    )

if "features_pen" in r:
    extra_losses.append(r["features_pen"])

In [14]:
sample_size = tf.cast(tf.shape(target)[0], tf.float32)

In [15]:
loss_weights = [0.1, 10]
for p, coef in zip(extra_losses, loss_weights):
    if coef != 0 and p is not None:
        p = coef * p * sample_size
        loss += p

In [16]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

3.518625

In [19]:
y = np.random.normal(size = (16000 * 10))

In [20]:
%%time

o = sess.run([r, logits, target, loss, optimizer], feed_dict = {X: [y, y], X_len: [len(y), len(y)]})

CPU times: user 37 s, sys: 13.2 s, total: 50.2 s
Wall time: 23.4 s


In [21]:
o[-2]

5618.202

In [22]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [23]:
!ls -lh test
!rm -rf test

total 711752
-rw-r--r--  1 huseinzolkepli  staff    77B Apr 27 08:14 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   326M Apr 27 08:14 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    72K Apr 27 08:14 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    19M Apr 27 08:14 model.ckpt.meta
